# 定义常量

In [2]:
import os, sys
import torch, json

from main import build_model_main
from util.slconfig import SLConfig
from util.visualizer import COCOVisualizer
from util import box_ops

model_config_path = "/home/wong/PycharmProjects/DINO/config/DINO/DINO_4scale_city.py"  # change the path of the model config file
model_checkpoint_path = "/home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/checkpoint_best_regular.pth"  # change the path of the model checkpoint
output_path = "/home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction.json"
output_path_with_name = "/home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction_with_name.json"
output_path_mask_with_name = "/home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction_mask_with_name.json"

anno_files_path = "/home/wong/Documents/DataSet/CityPerson/annotations/val_gt.json"
image_files_dir = "/home/wong/Documents/DataSet/CityPerson/test/"
label_names_json_path = "util/coco_city_id2name.json"

In [3]:
args = SLConfig.fromfile(model_config_path)
args.device = 'cuda'
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

/home/wong/miniconda3/envs/detr/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wong/miniconda3/envs/detr/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# load coco names
with open(label_names_json_path) as f:
    id2name = json.load(f)
    id2name = {int(k): v for k, v in id2name.items()}

In [5]:
# load test annotations
with open(anno_files_path) as f:
    anno_test = json.load(f)

In [6]:
anno_test["annotations"] = []

In [7]:
[k for k in anno_test]

['categories', 'images', 'annotations']

In [8]:
from PIL import Image
import datasets.transforms as T


def make_anno_json(image_id, category_id, bbox, score):
    dic = {
        # "id": anno_id,
        "image_id": image_id,
        "category_id": category_id,
        # "iscrowd": 0,
        # "area": area,
        "bbox": bbox,
        # "segmentation": None,
        "score": score
    }
    return dic


def make_anno_json_with_name(image_id, im_name, category_id, bbox, score):
    dic = {
        # "id": anno_id,
        "image_id": image_id,
        "im_name": im_name,
        "category_id": category_id,
        # "iscrowd": 0,
        # "area": area,
        "bbox": bbox,
        # "segmentation": None,
        "score": score
    }
    return dic


def make_anno_json_only_pedestrian(image_id, category_id, bbox, score):
    dic = {
        # "id": anno_id,
        "image_id": image_id,
        "category_id": 1 if category_id != 0 else 0,
        # "iscrowd": 0,
        # "area": area,
        "bbox": bbox,
        # "segmentation": None,
        "score": score
    }
    return dic


def box_xyxy_to_x0y0wh(x):
    x0, y0, x1, y1 = x.unbind(-1)
    b = [x0, y0, x1-x0, y1-y0]
    return torch.stack(b, dim=-1)


def make_array_json_with_mask(anno, need_name=False):
    anno_array = []

    for item in anno_test["images"]:
        image = Image.open(image_files_dir + item['im_name']).convert("RGB")  # load image
        image_id = item['id']
        image_name = item['im_name']
        # print(f"predicting image_id={image_id}")

        transform = T.Compose([
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image_trans, _ = transform(image, None)

        output = model.cuda()(image_trans[None].cuda())
        output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

        thershold = 0.5  # set a thershold

        scores = output['scores']
        labels = output['labels']
        # boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        boxes = box_xyxy_to_x0y0wh(output['boxes'])
        select_mask = scores > thershold

        scores_masked = scores[select_mask]
        labels_masked = labels[select_mask]
        boxes_masked = boxes[select_mask]

        id_label = [int(it) for it in labels_masked]
        # name_label = [id2name[int(it)] for it in labels[select_mask]]

        # id_label = [int(it) for it in labels]

        box_res = boxes_masked * torch.tensor([image.width, image.height, image.width, image.height]).cuda()
        # box_res = boxes * torch.tensor([image.width, image.height, image.width, image.height]).cuda()
        box_res_int = box_res.clone().type(torch.int).cpu().numpy().tolist()

        for bbox, label, score in zip(box_res_int, id_label, scores_masked):
            # for bbox, label, score in zip(box_res_int, id_label, scores):
            area = (bbox[2] * bbox[3])
            if need_name:
                json_dict = make_anno_json_with_name(image_id=image_id, im_name=image_name, category_id=label,
                                                     bbox=bbox, score=score.item())
            else:
                json_dict = make_anno_json(image_id=image_id, category_id=label, bbox=bbox, score=score.item())
            anno_array.append(json_dict)

    return anno_array


def make_array_json(anno, need_name=False):
    anno_array = []

    for item in anno_test["images"]:
        image = Image.open(image_files_dir + item['im_name']).convert("RGB")  # load image
        image_id = item['id']
        image_name = item['im_name']
        # print(f"predicting image_id={image_id}")

        transform = T.Compose([
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image_trans, _ = transform(image, None)

        output = model.cuda()(image_trans[None].cuda())
        output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

        thershold = 0.5  # set a thershold

        scores = output['scores']
        labels = output['labels']
        # boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
        boxes = box_xyxy_to_x0y0wh(output['boxes'])
        # select_mask = scores > thershold

        # scores_masked = scores[select_mask]
        # labels_masked = labels[select_mask]
        # boxes_masked = boxes[select_mask]

        # id_label = [int(it) for it in labels_masked]
        # name_label = [id2name[int(it)] for it in labels[select_mask]]

        id_label = [int(it) for it in labels]

        # box_res = boxes[select_mask] * torch.tensor([image.width, image.height, image.width, image.height]).cuda()
        box_res = boxes * torch.tensor([image.width, image.height, image.width, image.height]).cuda()
        box_res_int = box_res.clone().type(torch.int).cpu().numpy().tolist()

        # for bbox, label, score in zip(box_res_int, id_label, scores_masked):
        for bbox, label, score in zip(box_res_int, id_label, scores):
            area = (bbox[2] * bbox[3])
            if need_name:
                json_dict = make_anno_json_with_name(image_id=image_id, im_name=image_name, category_id=label,
                                                     bbox=bbox, score=score.item())
            else:
                json_dict = make_anno_json(image_id=image_id, category_id=label, bbox=bbox, score=score.item())
            anno_array.append(json_dict)

    return anno_array

# test
# make_array_json(anno_test)

In [9]:
anno_test["annotations"] = make_array_json(anno_test)
print(len(anno_test["annotations"]))

/home/wong/miniconda3/envs/detr/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449183005/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


150000


In [10]:
dict_anno = {'annotations': anno_test["annotations"]}
dict_anno

{'annotations': [{'image_id': 1,
   'category_id': 1,
   'bbox': [1155, 374, 40, 97],
   'score': 0.8608832359313965},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1221, 376, 37, 93],
   'score': 0.8594487905502319},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1193, 377, 36, 88],
   'score': 0.8547564744949341},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [947, 407, 14, 33],
   'score': 0.5088239908218384},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1242, 384, 32, 82],
   'score': 0.3048301637172699},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [977, 389, 17, 40],
   'score': 0.23128890991210938},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1226, 380, 35, 87],
   'score': 0.2295653522014618},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1152, 380, 38, 90],
   'score': 0.19910767674446106},
  {'image_id': 1,
   'category_id': 1,
   'bbox': [1162, 380, 35, 89],
   'score': 0.18617574870586395},
  {'image_id': 1,
   'category_id': 1,
   'bbox'

In [11]:
print("Saving to {}".format(output_path))
with open(output_path, "w") as out_f:
    json.dump(dict_anno, out_f)

Saving to /home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction.json


In [12]:
json_with_name = make_array_json(anno_test, need_name=True)

In [13]:
dict_anno_with_name = {'annotations': json_with_name}
dict_anno_with_name

{'annotations': [{'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [1155, 374, 40, 97],
   'score': 0.8608832359313965},
  {'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [1221, 376, 37, 93],
   'score': 0.8594487905502319},
  {'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [1193, 377, 36, 88],
   'score': 0.8547564744949341},
  {'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [947, 407, 14, 33],
   'score': 0.5088239908218384},
  {'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [1242, 384, 32, 82],
   'score': 0.3048301637172699},
  {'image_id': 1,
   'im_name': 'frankfurt_000000_000294_leftImg8bit.png',
   'category_id': 1,
   'bbox': [977, 389, 17, 40],
   'score': 0.23128890991210938},
  {'image_id': 1,
  

In [14]:
print("Saving to {}".format(output_path_with_name))
with open(output_path_with_name, "w") as out_f:
    json.dump(dict_anno_with_name, out_f)

Saving to /home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction_with_name.json


In [15]:
json_mask_with_name = make_array_json_with_mask(anno_test, need_name=True)
dict_anno_mask_with_name = {'annotations': json_mask_with_name}
print("Saving to {}".format(output_path_with_name))
with open(output_path_mask_with_name, "w") as out_f:
    json.dump(dict_anno_mask_with_name, out_f)

Saving to /home/wong/Documents/DINO_logs/11-25-16-00(4scale_Res50_12epoch+binary)/custom_prediction_with_name.json
